# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [28]:
# imports
import panel as pn
pn.extension('plotly')
from panel.interact import interact
from panel import widgets
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import math

In [29]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [30]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

file_path2 = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path2, index_col="year")

san_francisco_averages_annual = sfo_data.groupby('year').mean()
avg_neighborhood_cost = sfo_data.groupby('neighborhood').mean()
top_10_expensive_neighborhood = avg_neighborhood_cost.nlargest(10, 'sale_price_sqr_foot')

avg_expensive = top_10_expensive_neighborhood.reset_index()

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path, index_col='Neighborhood')

avg_data_location = avg_neighborhood_cost.join(df_neighborhood_locations)
avg_data_location.reset_index(inplace=True)

avg_neighborhood_data = sfo_data.groupby(['year', 'neighborhood']).mean()

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path, index_col='Neighborhood')
avg_data_location = avg_neighborhood_cost.join(df_neighborhood_locations)
avg_data_location.reset_index(inplace=True)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [41]:
# Define Panel Visualization Functions
def housing_units_per_year():
    return san_francisco_averages_annual.hvplot.bar(
        x='year',
        y='housing_units', 
        ylim=(350000, 400000),
        dynamic=False
    )

def average_gross_rent():
    return san_francisco_averages_annual['gross_rent'].hvplot.line(
        x='year', 
        y='gross_rent',
        title='Average Gross Rent in San Francisco(Annual)',
        dynamic=False
    )

def average_sales_price():
    return san_francisco_averages_annual['sale_price_sqr_foot'].hvplot.line(
        x='year', 
        y='sale_price_sqr_foot', 
        title='Average Sales Price in San Francisco(Annual)',
        dynamic=False
    )


def average_price_by_neighborhood():
    return avg_neighborhood_data.hvplot.bar(
        x='year',
        y='sale_price_sqr_foot', 
        groupby='neighborhood', 
        widget_location='top', 
        title='Average Prices Per Neighborhood',
        dynamic=False
    )

def top_most_expensive_neighborhoods():
    return top_10_expensive_neighborhood.hvplot(
        x='neighborhood', 
        y='sale_price_sqr_foot', 
        kind='bar', rot=45, 
        title='Top 10 Most Expensive Neighborhoods',
        dynamic=False
    )



def parallel_coordinates():
    return px.parallel_coordinates(avg_expensive, 
                                   color='sale_price_sqr_foot')


def parallel_categories():
    return px.parallel_categories(
        avg_expensive,
        dimensions=['neighborhood', 
                    'sale_price_sqr_foot', 
                    'housing_units', 
                    'gross_rent'],
        color='sale_price_sqr_foot'
    )


def neighborhood_map():
    return px.scatter_mapbox(
    avg_data_location,
    lat="Lat",
    lon="Lon",
    color="neighborhood"
)



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [42]:
map_column = pn.Column(
    "## Maps",
    neighborhood_map()
)
bar_chart_column = pn.Column(
    "## Bar Charts",
    housing_units_per_year(),
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods(),
)
parallel_column = pn.Column(
    "## Parallels",
    parallel_coordinates(),
    parallel_categories()
)
line_chart_column = pn.Column(
    average_gross_rent(),
    average_sales_price()
)

san_francisco_housing_dashboard = pn.Tabs(
    ('Maps', map_column), 
    ('Bar Charts', bar_chart_column),
    ('Parallels', parallel_column),
    ('Line Charts', line_chart_column)
)

## Serve the Panel Dashboard

In [43]:
san_francisco_housing_dashboard.servable()

Traceback (most recent call last):
 File "C:\Users\miche\anaconda3\lib\site-packages\pyviz_comms\__init__.py", line 325, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\miche\anaconda3\lib\site-packages\panel\viewable.py", line 241, in _on_msg
 patch.apply_to_document(doc, comm.id)
 File "C:\Users\miche\anaconda3\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 100, in apply_to_document
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\miche\anaconda3\lib\site-packages\bokeh\document\document.py", line 1150, in _with_self_as_curdoc
 return f()
 File "C:\Users\miche\anaconda3\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 100, in <lambda>
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\miche\anaconda3\lib\site-packages\bokeh\document\document.py", line 398, in apply_json_patch
 self._trigger_on_message(event_json["msg_type"], event_json["msg_data"])
 File "C:\Users\miche\anaconda3\lib\site-packages\bokeh\document\document.py", line 687, in _trigger_on_message
 cb(msg_data)
 File "C:\Users\miche\anaconda3\lib\site-packages\bokeh\document\document.py", line 348, in apply_json_event
 event = Event.decode_json(json)
 File "C:\Users\miche\anaconda3\lib\site-packages\bokeh\events.py", line 167, in decode_json
 raise ValueError("Could not find appropriate Event class for event_name: %r" % event_name)
ValueError: Could not find appropriate Event class for event_name: 'document_ready'

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] Column
            [0] Row
                [0] HSpacer()
                [1] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [2] HSpacer()
            [1] HoloViews(HoloMap, widget_location='top')
        [3] HoloViews(Bars)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [3] Column
        [0] HoloViews(Curve)
        [1] HoloViews(Curve)